In [1]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
import re
from fake_useragent import UserAgent

# CGV

## Event List

In [2]:
def get_CGV_GoodsIdx(event_idx):
    url = "https://m.cgv.co.kr/Event/GiveawayEventDetail.aspx/GetGiveawayEventDetail"
    body = {
        "eventIndex": event_idx,
        "giveawayIndex": None
    }
    response = requests.post(url, json=body)
    text = response.text
    loads = json.loads(text)
    goods_idx = loads["d"]["GiveawayItemList"][0]["GiveawayItemCode"]
    return goods_idx

def get_CGV_EventList(movie_filter='', goods_filter=''):
    url = "https://m.cgv.co.kr/Event/GiveawayEventList.aspx/GetGiveawayEventList"
    body = {
        "theaterCode": "",
        "pageNumber": "1",
        "pageSize": "100"
    }
    response = requests.post(url, json=body)
    text = response.text
    loads = json.loads(text)
    soup = BeautifulSoup(loads['d']['List'])
    event_list = soup.find_all("li")
    event_df = pd.DataFrame()
    p = re.compile(r'\[([\d\D]+)\]') # 영화이름 추출 정규식
    for event in event_list:
        event_idx = event['data-eventidx']
        goods_idx = get_CGV_GoodsIdx(event_idx)
        event_name = event.find("strong").get_text()
        event_period = event.find("span").get_text()
        m = p.match(event_name)
        if m is None:
            movie_name = event_name
            goods_name = event_name
        else:
            movie_name = m.group()[1:-1]
            goods_name = event_name[m.end():] # 띄어쓰기 했다안했다
        event_dict = {"event_idx":event_idx, "goods_idx":goods_idx, "movie":movie_name, "goods":goods_name, "period":event_period}
        event_df = pd.concat([event_df, pd.DataFrame.from_dict(event_dict, orient='index').T], ignore_index=True)
    event_df = event_df[
        (event_df["movie"].str.contains(movie_filter)) &
        (event_df["goods"].str.contains(goods_filter))
    ]
    # 날짜 필터도 추가 예정
    # 위에서 필터링
    return event_df

In [3]:
event_df = get_CGV_EventList()
event_df

,event_idx,goods_idx,movie,goods,period
0,202506236274,2025062362746226,F1 더 무비,4DX 포스터,2025.06.25 ~ 2025.07.11
1,202506236273,2025062362736225,F1 더 무비,IMAX 포스터,2025.06.25 ~ 2025.07.22
2,202506236278,2025062362786229,F1 더 무비,SX 와이드 포스터,2025.06.30 ~ 2025.07.11
3,202506236275,2025062362756227,F1 더 무비,SX 포스터,2025.06.25 ~ 2025.07.11
4,202506186257,2025061862576209,F1 더 무비,TTT,2025.06.25 ~ 2025.07.24
5,202506236276,2025062362766228,F1 더 무비,ULTRA 4DX 모먼트라벨,2025.06.30 ~ 2025.07.11
6,202507036329,2025070363296272,SMG 무비갤러리,하이큐 승자와 패자_북마크,2025.07.04 ~ 2025.07.10
7,202506266284,2025062662846234,SMG무비갤러리,블리치 지옥편 포스터,2025.06.26 ~ 2025.07.17
8,202507036328,2025070363286271,SMG무비갤러리,하이큐 끝과 시작_북마크,2025.07.04 ~ 2025.07.10
9,202506056186,2025060561866145,SMG무비갤러리,블리치 페이드투블랙 포스터,2025.06.06 ~ 2025.07.17


## Goods

- type2: 재고 보유 가능성 높음
- type2.5: 재고 소진 중
- type3: 재고 소진 임박
- type4: 마감

In [4]:
def get_CGV_Theaters(giveawayIndex):
    url = "https://m.cgv.co.kr/Event/GiveawayEventDetail.aspx/GetGiveawayTheaterInfo"
    body = {
        "giveawayIndex": giveawayIndex,
        "areaCode": None
    }
    response = requests.post(url, json=body)
    text = response.text
    loads = json.loads(text)
    AreaList = loads["d"]["AreaList"]
    Theaters = pd.DataFrame()
    for Area in AreaList:
        AreaCode = Area["AreaCode"]
        body = {
            "giveawayIndex": giveawayIndex,
             "areaCode": AreaCode
        }
        response = requests.post(url, json=body)
        text = response.text
        loads = json.loads(text)
        Theater = pd.DataFrame(loads["d"]["TheaterList"])
        Theaters = pd.concat([Theaters, Theater[["TheaterName", "TheaterCode", "CountTypeCode"]]], ignore_index=True)
    return Theaters

In [7]:
giveawayIndex = '2025062362746226'
Theaters = get_CGV_Theaters(giveawayIndex)
Theaters

,TheaterName,TheaterCode,CountTypeCode
0,CGV 강변,0001,type3
1,CGV 고덕강일,0366,type2
2,CGV 방학,0288,type2.5
3,CGV 상봉,0046,type4
4,CGV 신촌아트레온,0150,type2
5,CGV 여의도,0112,type4
6,CGV 연남,0292,type4
7,CGV 영등포,0059,type4
8,CGV 왕십리,0074,type4
9,CGV 용산아이파크몰,0013,type4


In [8]:
def Get_CGV_GoodsStatus(movie_filter="", goods_filter=""):
    event_df = get_CGV_EventList(movie_filter=movie_filter, goods_filter=goods_filter)
    GoodsStatus = pd.DataFrame()
    for idx in event_df.index:
        giveawayIndex = event_df.loc[idx]["goods_idx"]
        Theaters = get_CGV_Theaters(giveawayIndex)
        Theaters["movie"] =  event_df.loc[idx]["movie"]
        Theaters["goods"] =  event_df.loc[idx]["goods"]
        Theaters["period"] =  event_df.loc[idx]["period"]
        GoodsStatus = pd.concat([GoodsStatus, Theaters], ignore_index=True)
    return GoodsStatus

In [10]:
GoodsStatus = Get_CGV_GoodsStatus(movie_filter="F1 더 무비", goods_filter="포스터")
GoodsStatus

,TheaterName,TheaterCode,CountTypeCode,movie,goods,period
0,CGV 강변,0001,type3,F1 더 무비,4DX 포스터,2025.06.25 ~ 2025.07.11
1,CGV 고덕강일,0366,type2,F1 더 무비,4DX 포스터,2025.06.25 ~ 2025.07.11
2,CGV 방학,0288,type2.5,F1 더 무비,4DX 포스터,2025.06.25 ~ 2025.07.11
3,CGV 상봉,0046,type4,F1 더 무비,4DX 포스터,2025.06.25 ~ 2025.07.11
4,CGV 신촌아트레온,0150,type2,F1 더 무비,4DX 포스터,2025.06.25 ~ 2025.07.11
...,...,...,...,...,...,...
91,CGV 천안펜타포트,0110,type2,F1 더 무비,SX 포스터,2025.06.25 ~ 2025.07.11
92,CGV 서면,0005,type2,F1 더 무비,SX 포스터,2025.06.25 ~ 2025.07.11
93,CGV 아시아드,0160,type4,F1 더 무비,SX 포스터,2025.06.25 ~ 2025.07.11
94,CGV 울산삼산,0128,type2,F1 더 무비,SX 포스터,2025.06.25 ~ 2025.07.11


# LotteCinema

In [11]:
def get_LC_EventList():
    url = "https://event.lottecinema.co.kr/LCWS/Event/EventData.aspx"
    # 특별기획전 "EventClassificationCode":"10"
    body = {
        "ParamList": '{"MethodName":"GetEventLists","channelType":"","osType":"","osVersion":"","EventClassificationCode":"20","SearchText":"","CinemaID":"","PageNo":1,"PageSize":100,"MemberNo":"0"}'
    }
    response = requests.post(url, data=body)
    text = response.text
    loads = json.loads(text)
    event_df = pd.DataFrame(loads["Items"])

    goods_info = [get_LC_GoodsInfo(x) for x in event_df['EventID']]
    event_df = pd.concat([event_df[["EventID", "EventName", "ProgressStartDate", "ProgressEndDate"]], 
                           pd.json_normalize(goods_info)[["FrGiftID", "FrGiftNm"]]],
                         axis=1)

    event_df = event_df[~event_df['FrGiftID'].isna()].reset_index(drop=True)

    p = re.compile(r'\<([\d\D]+)\>') # 영화이름 추출 정규식
    event_df["movie_name"] = [
        '' if p.match(x) is None else p.match(x).group()[1:-1] 
        for x in event_df['EventName']
    ]

    p = re.compile(r'\,([\d\D]+)\)') # 굿즈이름 추출 정규식
    event_df["goods_name"] = [
        '' if len(p.findall(x))==0 else p.findall(x)[0].replace(' ', '')
        for x in event_df['FrGiftNm']
    ]
    return event_df

def get_LC_GoodsInfo(event_idx):
    # EventID, FrGiftID, FrGiftNm
    url = "https://event.lottecinema.co.kr/LCWS/Event/EventData.aspx"
    body = {
        "ParamList": '{"MethodName":"GetInfomationDeliveryEventDetail","channelType":"","osType":"","osVersion":"","EventID":%s}' % (event_idx)
    }
    response = requests.post(url, data=body)
    text = response.text
    loads = json.loads(text)
    goods_info = loads["InfomationDeliveryEventDetail"][0]["GoodsGiftItems"]
    if len(goods_info) == 0: 
        goods_info = None
    else:
        goods_info = goods_info[0]
    return goods_info

In [35]:
import requests
import json
import pandas as pd
import re

def get_LC_GoodsInfo(event_idx):
    url = "https://www.lottecinema.co.kr/LCWS/Event/EventData.aspx"
    param_json = {
        "MethodName": "GetInfomationDeliveryEventDetail",
        "channelType": "HO",
        "osType": "W",
        "osVersion": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
        "EventID": str(event_idx)
    }
    files = {
        "paramList": (None, json.dumps(param_json), "application/json")
    }

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
        "Referer": "https://www.lottecinema.co.kr/NLCHS/Event",
        "Origin": "https://www.lottecinema.co.kr",
        "Accept": "application/json, text/plain, */*",
        "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
        "Connection": "keep-alive",
    }

    with requests.Session() as session:
        # 세션 쿠키 미리 받기
        session.get("https://www.lottecinema.co.kr/NLCHS/Event", headers=headers)
        response = session.post(url, files=files, headers=headers)
        response.raise_for_status()
        data = response.json()
    print(data)

    goods_info = data["InfomationDeliveryEventDetail"][0]["GoodsGiftItems"]
    if not goods_info:
        return None
    return goods_info[0]

def get_LC_EventList(special_exhibition=False):
    url = "https://www.lottecinema.co.kr/LCWS/Event/EventData.aspx"
    param_json = {
        "MethodName": "GetEventLists",
        "channelType": "HO",
        "osType": "W",
        "osVersion": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
        "EventClassificationCode": "20",
        "SearchText": "",
        "CinemaID": "",
        "PageNo": 1,
        "PageSize": 100,
        "MemberNo": "0"
    }
    if special_exhibition:
        param_json["EventClassificationCode"] = "10"
    files = {
        "paramList": (None, json.dumps(param_json), "application/json")
    }

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
        "Referer": "https://www.lottecinema.co.kr/NLCHS/Event",
        "Origin": "https://www.lottecinema.co.kr",
        "Accept": "application/json, text/plain, */*",
        "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
        "Connection": "keep-alive",
    }

    with requests.Session() as session:
        session.get("https://www.lottecinema.co.kr/NLCHS/Event", headers=headers)
        response = session.post(url, files=files, headers=headers)
        response.raise_for_status()
        data = response.json()

    event_df = pd.DataFrame(data["Items"])
    print(event_df)
    goods_info = [get_LC_GoodsInfo(x) for x in event_df['EventID']]
    goods_df = pd.json_normalize([g if g else {} for g in goods_info])
    # 결측값 방지
    goods_df = goods_df.reindex(columns=["FrGiftID", "FrGiftNm"]).fillna('')

    event_df = pd.concat(
        [event_df[["EventID", "EventName", "ProgressStartDate", "ProgressEndDate"]], goods_df],
        axis=1
    )

    event_df = event_df[event_df['FrGiftID'] != ''].reset_index(drop=True)

    # 영화 이름 추출 (<> 안 내용)
    p_movie = re.compile(r'<([^<>]+)>')
    event_df["movie_name"] = event_df['EventName'].apply(lambda x: p_movie.search(x).group(1) if p_movie.search(x) else '')

    # 굿즈 이름 추출 (, ... ) 형태에서 추출
    p_goods = re.compile(r',([^,]+)\)')
    event_df["goods_name"] = event_df['FrGiftNm'].apply(lambda x: p_goods.search(x).group(1).strip() if p_goods.search(x) else '')

    return event_df


In [29]:
special_EventList = get_LC_EventList(special_exhibition = True)

            EventID                          EventName  \
0   101010048025006             스코시즘 라이브뷰잉 <여름방학 보충수업>   
1   101010021225013                        도라에몽 피규어 런칭   
2   101010021225014                     킹 오브 킹스 마그넷 런칭   
3   101110038525015                  <인디시네마> 7월 특별 시사회   
4   101010024725007            [전지적 독자 시점] 얼리버드 특가 이벤트   
5   101010046225004                  이 달의 광음시네마 7월 라인업   
6   101010021225012                        쥬라기월드 굿즈 런칭   
7   101010038525018           [안산지역] 모든 영화를 5년 전 가격으로!   
8   101010038525019           [대구지역] 모든 영화를 5년 전 가격으로!   
9   101010024725006  <극장판 도라에몽: 진구의 그림이야기> 관람권 얼리버드 특가   
10  101110038525012                        광음시네마 FESTA   
11  101010038525013              어린이ㆍ청소년 영화교육 프로그램 너랑봄   
12  101010049125002                      롯데시네마 대관단체 안내   
13  101010047625010               ♥ 스피드 데이팅 2기 모집 안내 ♥   
14  101170014224002                   2025 VIP 특별 제휴혜택   
15  101010014224034              2025 롯데시네마 VIP 멤버십 안내   
16  1010100476

In [36]:
test = get_LC_GoodsInfo("101110038525012")
print(test)

{'InfomationDeliveryEventDetail': [{'ButtonSetting': [{'ConfigureButtonCode': '30', 'ConfigureButtonName': '응모하기', 'ConfigureButtonAsName': '쿠폰받기', 'ConfigureButtonValue1': '', 'ConfigureButtonValue2': '', 'ConfigureButtonValue3': '', 'ConfigureButtonValue4': '', 'ConfigureButtonValue5': '', 'NeedCinemaTalkCheckYN': None}, {'ConfigureButtonCode': '50', 'ConfigureButtonName': '자세히보기', 'ConfigureButtonAsName': '광음시네마 예매하기', 'ConfigureButtonValue1': 'https://www.lottecinema.co.kr/NLCHS/Cinema/SpecialCinema?divisionCode=2&screendivcd=901', 'ConfigureButtonValue2': '', 'ConfigureButtonValue3': '', 'ConfigureButtonValue4': '', 'ConfigureButtonValue5': '', 'NeedCinemaTalkCheckYN': None}], 'JoinStatus': [], 'GoodsGiftItems': [], 'EventClassificationCode': '10', 'EventID': '101110038525012', 'EventName': '광음시네마 FESTA', 'ProgressStartDate': '2025-06-25', 'ProgressEndDate': '2025-07-25', 'WinnerAnnouncmentDate': '2025-08-07', 'ImgUrl': 'http://cf.lottecinema.co.kr//Media/Event/f3e9065cfd2741a1ada

In [27]:
special_EventList

,EventID,EventName,ProgressStartDate,ProgressEndDate,FrGiftID,FrGiftNm,movie_name,goods_name


In [30]:
LC_EventList = get_LC_EventList()
# LC_EventList

            EventID                                  EventName  \
0   201010048025172             롯시네 보석 발굴 프로젝트 NO.22 <셔터 아일랜드>   
1   201010016925362                             <킹 오브 킹스> 무비티켓   
2   201010048025159             <극장판 도라에몽: 진구의 그림이야기> 프리미어 상영회   
3   201010016925361                         <시네마천국>2주차 주말증정이벤트   
4   201010048025170             <영화 코바야시네 메이드래곤> 특수관 굿즈 증정 이벤트   
5   201010016925358                           <쥬라기월드>2주차 증정이벤트   
6   201010016925354                            <우교시>1주차현장증정이벤트   
7   201010048025163  <영화 코바야시네 메이드래곤 외로움쟁이 용> SPECIAL ART CARD   
8   201110038525025               <전지적 독자 시점> 얼리버드 프로모션 티켓편 2차   
9   201010016925355                          <시네마 천국>2주차 증정이벤트   
10  201010016925356                        <발코니의 여자들>1주차 증정이벤트   
11  201010016925357                         <미션임파서블8>9주차 증정이벤트   
12  201010016925350                            <슈퍼맨> 시그니처 아트카드   
13  201010016925360                                <BTS> 상영 고지   
14  201110

In [17]:
import requests
import json

url = "https://www.lottecinema.co.kr/LCWS/Event/EventData.aspx"

param_json = {
    "MethodName": "GetEventLists",
    "channelType": "HO",
    "osType": "W",
    "osVersion": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    "EventClassificationCode": "30",
    "SearchText": "",
    "CinemaID": "",
    "PageNo": 1,
    "PageSize": 4,
    "MemberNo": "0"
}

files = {
    "paramList": (None, json.dumps(param_json), "application/json")
}

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    "Referer": "https://www.lottecinema.co.kr/NLCHS/Event",
    "Origin": "https://www.lottecinema.co.kr",
    "Accept": "application/json, text/plain, */*",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Connection": "keep-alive",
    # **Content-Type는 지정하지 마세요!**
}

with requests.Session() as session:
    session.get("https://www.lottecinema.co.kr/NLCHS/Event", headers=headers)
    response = session.post(url, files=files, headers=headers)
    print("Status Code:", response.status_code)
    data = response.json()
    print(json.dumps(data, indent=2, ensure_ascii=False))


Status Code: 200
{
  "Items": [
    {
      "EventID": "301010038525001",
      "EventName": "[안산, 대구지역] 요금인하 안내",
      "EventClassificationCode": "30",
      "EventTypeCode": "101",
      "EventTypeName": "정보전달형(공지)",
      "ProgressStartDate": "2025.07.01",
      "ProgressEndDate": "2025.12.30",
      "ImageUrl": "",
      "ImageAlt": "[안산, 대구지역] 요금인하 안내",
      "ImageDivisionCode": 0,
      "CinemaID": "5012",
      "CinemaName": "대구광장 외",
      "CinemaAreaCode": "5",
      "CinemaAreaName": "경북/대구",
      "DevTemplateYN": 0,
      "CloseNearYN": 0,
      "RemainsDayCount": 172,
      "EventWinnerYN": 0,
      "EventSeq": 4,
      "EventCntnt": "● 기간 : 2025년 07월 01일～<BR>● 대상영화관 : 롯데시네마 센트럴락, 안산, 안산고잔 / 대구광장, 상인, 성서 <BR>● 내용 :&nbsp;조조 - 성인 1만원, 청소년 8천원 관람 / 주중, 주말 - 성인 12,000원, 청소년 9천원 관람<BR>",
      "EventNtc": "<P>* 현장, 온라인 예매 시 2D영화에 한하여 인하된 금액으로 자동 적용됩니다.</P>\r\n<P>* 특수관, 특수좌석(수퍼플렉스, 씨네커플 등) 및 컨텐츠에 따라 추가요금이 발생할 수 있습니다. <BR>* 안산관 샤롯데관은 주중 22,000원 / 주말 25,000원 관람 가능합니다. <BR>* 장애인,

In [15]:
import requests
import json

url = "https://www.lottecinema.co.kr/LCWS/Event/EventData.aspx"

# 실제 브라우저에서 보내는 paramList JSON
param_json = {
    "MethodName": "GetEventLists",
    "channelType": "HO",
    "osType": "W",
    "osVersion": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36",
    "EventClassificationCode": "30",
    "SearchText": "",
    "CinemaID": "",
    "PageNo": 1,
    "PageSize": 4,
    "MemberNo": "0"
}

# multipart/form-data 형식으로 paramList 필드에 JSON 문자열 넣기
files = {
    "paramList": (None, json.dumps(param_json), "application/json")
}

# 브라우저 요청 헤더를 최대한 흉내 내기
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36",
    "Referer": "https://www.lottecinema.co.kr/NLCHS/Event",
    "Origin": "https://www.lottecinema.co.kr",
    "Accept": "application/json, text/plain, */*",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Connection": "keep-alive",
}

with requests.Session() as session:
    # 필요 시 먼저 이벤트 페이지 방문해 세션 쿠키 확보
    session.get("https://www.lottecinema.co.kr/NLCHS/Event", headers=headers)

    # POST 요청
    response = session.post(url, files=files, headers=headers)

    print("Status Code:", response.status_code)
    # json 응답 파싱
    data = response.json()
    print(json.dumps(data, indent=2, ensure_ascii=False))


Status Code: 200
{
  "Items": [
    {
      "EventID": "301010038525001",
      "EventName": "[안산, 대구지역] 요금인하 안내",
      "EventClassificationCode": "30",
      "EventTypeCode": "101",
      "EventTypeName": "정보전달형(공지)",
      "ProgressStartDate": "2025.07.01",
      "ProgressEndDate": "2025.12.30",
      "ImageUrl": "",
      "ImageAlt": "[안산, 대구지역] 요금인하 안내",
      "ImageDivisionCode": 0,
      "CinemaID": "5012",
      "CinemaName": "대구광장 외",
      "CinemaAreaCode": "5",
      "CinemaAreaName": "경북/대구",
      "DevTemplateYN": 0,
      "CloseNearYN": 0,
      "RemainsDayCount": 172,
      "EventWinnerYN": 0,
      "EventSeq": 4,
      "EventCntnt": "● 기간 : 2025년 07월 01일～<BR>● 대상영화관 : 롯데시네마 센트럴락, 안산, 안산고잔 / 대구광장, 상인, 성서 <BR>● 내용 :&nbsp;조조 - 성인 1만원, 청소년 8천원 관람 / 주중, 주말 - 성인 12,000원, 청소년 9천원 관람<BR>",
      "EventNtc": "<P>* 현장, 온라인 예매 시 2D영화에 한하여 인하된 금액으로 자동 적용됩니다.</P>\r\n<P>* 특수관, 특수좌석(수퍼플렉스, 씨네커플 등) 및 컨텐츠에 따라 추가요금이 발생할 수 있습니다. <BR>* 안산관 샤롯데관은 주중 22,000원 / 주말 25,000원 관람 가능합니다. <BR>* 장애인,

In [2]:
import requests
import json

session = requests.Session()

# 1. 메인 페이지 접속 (쿠키 받아오기)
resp_main = session.get("https://www.lottecinema.co.kr")
print("받은 쿠키:", session.cookies.get_dict())

# 2. API 요청 준비
url = "https://www.lottecinema.co.kr/LCWS/Event/EventData.aspx"

param_json = {
    "MethodName": "GetInfomationDeliveryEventDetail",
    "channelType": "HO",
    "osType": "W",
    "osVersion": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    "EventID": "201010016925350"
}

files = {
    "paramList": (
        None,
        json.dumps(param_json),
        "application/json"
    )
}

headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://www.lottecinema.co.kr/NLCHS/Event/EventTemplateInfo?eventId=201010016925350"
}

# 3. 세션 유지된 상태로 POST 요청
response = session.post(url, files=files, headers=headers)

print("Status Code:", response.status_code)
print("Response Text:", response.text[:500])


받은 쿠키: {}
Status Code: 200
Response Text: {"InfomationDeliveryEventDetail":[{"ButtonSetting":[],"JoinStatus":[],"GoodsGiftItems":[{"EventID":"201010016925350","FrGiftID":"12526","FrGiftNm":"<슈퍼맨>시그니처 아트카드(7/9수~22화,회차당 수령제한O)"}],"EventClassificationCode":"20","EventID":"201010016925350","EventName":"<슈퍼맨> 시그니처 아트카드","ProgressStartDate":"2025-07-09","ProgressEndDate":"2025-07-22","WinnerAnnouncmentDate":"2025-07-23","ImgUrl":"http://cf.lottecinema.co.kr//Media/Event/44358f22343a42598bc2be297940687c.jpg","ImgAlt":"Signature\r\nArt Card No.


In [61]:
def get_LC_Theaters(EventID, FrGiftID):
    url = "https://event.lottecinema.co.kr/LCWS/Event/EventData.aspx"
    body = {
        "ParamList": '{"MethodName":"GetCinemaGoods","channelType":"","osType":"","osVersion":"","EventID":%s, "GiftID":%s}' % (EventID, FrGiftID)
    }
    response = requests.post(url, data=body)
    text = response.text
    loads = json.loads(text)
    Theaters = pd.DataFrame(loads["CinemaDivisionGoods"])
    return Theaters

In [62]:
EventID = "201010016924197"
FrGiftID = "9190"
Theaters = get_LC_Theaters(EventID, FrGiftID)
Theaters

,DivisionCode,DetailDivisionCode,CinemaID,CinemaNameKR,CinemaNameUS,SortSequence,Cnt,DetailDivisionNameKR,DetailDivisionNameUS
0,1,0001,1004,건대입구,Konkuk Univ,6,20,서울,Seoul
1,1,0001,1009,김포공항,Gimpo Int'l Airport,7,20,서울,Seoul
2,1,0001,1003,노원,Nowon,8,49,서울,Seoul
3,1,0001,1007,신림,Sillim,18,50,서울,Seoul
4,1,0001,1002,영등포,Yeongdeungpo,20,50,서울,Seoul
5,1,0001,1016,월드타워,WorldTower,22,50,서울,Seoul
6,1,0001,1008,청량리,Cheongnyangni,25,40,서울,Seoul
7,1,0002,3008,부평역사,Bupyeong Sation,16,50,경기/인천,Gyeonggi/Incheon
8,1,0002,3018,평촌(범계역),Pyeongchon,46,50,경기/인천,Gyeonggi/Incheon
9,1,0004,6004,수완(아울렛),Suwan,71,50,전라/광주,Cholla/Gwangju


In [63]:
def Get_LC_GoodsStatus(movie_filter="", goods_filter=""):
    event_df = get_LC_EventList()
    event_df = event_df[
        (event_df["EventName"].str.contains(movie_filter)) &
        (event_df["FrGiftNm"].str.contains(goods_filter))
    ]
    GoodsStatus = pd.DataFrame()
    for idx in event_df.index:
        EventID = event_df.loc[idx]["EventID"]
        FrGiftID = event_df.loc[idx]["FrGiftID"]
        Theaters = get_LC_Theaters(EventID, FrGiftID)
        Theaters["movie"] =  event_df.loc[idx]["movie_name"]
        Theaters["goods"] =  event_df.loc[idx]["goods_name"]
        Theaters["period"] =  event_df.loc[idx]["ProgressStartDate"] + '~' + event_df.loc[idx]["ProgressEndDate"]
        GoodsStatus = pd.concat([GoodsStatus, Theaters], ignore_index=True)
    return GoodsStatus

In [64]:
Get_LC_GoodsStatus(goods_filter='엽서')

,DivisionCode,DetailDivisionCode,CinemaID,CinemaNameKR,CinemaNameUS,SortSequence,Cnt,DetailDivisionNameKR,DetailDivisionNameUS,movie,goods,period
0,1,0001,1004,건대입구,Konkuk Univ,6,10,서울,Seoul,범죄도시4,친필엽서세트,2024.05.01~2024.05.07
1,1,0001,1009,김포공항,Gimpo Int'l Airport,7,50,서울,Seoul,범죄도시4,친필엽서세트,2024.05.01~2024.05.07
2,1,0001,1016,월드타워,WorldTower,22,0,서울,Seoul,범죄도시4,친필엽서세트,2024.05.01~2024.05.07
3,1,0002,3024,수원(수원역),Suwon,23,10,경기/인천,Gyeonggi/Incheon,범죄도시4,친필엽서세트,2024.05.01~2024.05.07
4,1,0003,4008,대전센트럴,Daejeon Central,5,20,충청/대전,Chungcheong/Daejeon,범죄도시4,친필엽서세트,2024.05.01~2024.05.07
5,1,0004,6002,전주(백화점),Jeonju,109,40,전라/광주,Cholla/Gwangju,범죄도시4,친필엽서세트,2024.05.01~2024.05.07
6,1,0005,5006,대구율하,Daegu Yulha,6,49,경북/대구,Gyeongbuk/Daegu,범죄도시4,친필엽서세트,2024.05.01~2024.05.07
7,1,0101,2009,광복,Gwangbok,3,50,경남/부산/울산,Gyeongnam/Busan/Ulsan,범죄도시4,친필엽서세트,2024.05.01~2024.05.07
8,1,0101,2004,부산본점,Busan,11,50,경남/부산/울산,Gyeongnam/Busan/Ulsan,범죄도시4,친필엽서세트,2024.05.01~2024.05.07
9,1,0101,5001,울산(백화점),Ulsan,19,50,경남/부산/울산,Gyeongnam/Busan/Ulsan,범죄도시4,친필엽서세트,2024.05.01~2024.05.07


#### 번외 수원 기획전 

In [65]:
EventID = "101010014224011"
url = "https://event.lottecinema.co.kr/LCWS/Event/EventData.aspx"
body = {
    "ParamList": '{"MethodName":"GetInfomationDeliveryEventDetail","channelType":"","osType":"","osVersion":"","EventID":%s}' % (EventID)
}
response = requests.post(url, data=body)
text = response.text
loads = json.loads(text)
suwon_list = loads["InfomationDeliveryEventDetail"][0]["GoodsGiftItems"]

suwon_list2 = []
for suwon in suwon_list:
    EventID = suwon["EventID"]
    FrGiftID = suwon["FrGiftID"]
    url = "https://event.lottecinema.co.kr/LCWS/Event/EventData.aspx"
    body = {
        "ParamList": '{"MethodName":"GetCinemaGoods","channelType":"","osType":"","osVersion":"","EventID":%s, "GiftID":%s}' % (EventID, FrGiftID)
    }
    response = requests.post(url, data=body)
    text = response.text
    loads = json.loads(text)
    suwon.update(loads["CinemaDivisionGoods"][0])
    suwon_list2.append(suwon)
    #     print(loads["CinemaDivisionGoods"])
pd.DataFrame(suwon_list2)

,EventID,FrGiftID,FrGiftNm,DivisionCode,DetailDivisionCode,CinemaID,CinemaNameKR,CinemaNameUS,SortSequence,Cnt,DetailDivisionNameKR,DetailDivisionNameUS
0,101010014224011,9183,수원 특별 기획전(스파이더맨 어크로스 더 유니버스),1,0002,3024,수원(수원역),Suwon,23,50,경기/인천,Gyeonggi/Incheon
1,101010014224011,9184,수원 특별 기획전(스파이더맨 노웨이 홈 FUN),1,0002,3024,수원(수원역),Suwon,23,100,경기/인천,Gyeonggi/Incheon
2,101010014224011,9185,수원 특별 기획전(그란투리스모),1,0002,3024,수원(수원역),Suwon,23,350,경기/인천,Gyeonggi/Incheon
3,101010014224011,9186,수원 특별 기획전(탑건: 매버릭),1,0002,3024,수원(수원역),Suwon,23,850,경기/인천,Gyeonggi/Incheon
4,101010014224011,9187,수원 특별 기획전(언어의 정원),1,0002,3024,수원(수원역),Suwon,23,350,경기/인천,Gyeonggi/Incheon
5,101010014224011,9188,수원 특별 기획전(날씨의 아이),1,0002,3024,수원(수원역),Suwon,23,100,경기/인천,Gyeonggi/Incheon


# Megabox

In [71]:
def get_MB_EventList():
    url = "https://www.megabox.co.kr/on/oh/ohe/Event/eventMngDiv.do"
    body = {
        "currentPage": "1",
        "recordCountPerPage": "1000",
        "eventStatCd": "ONG",
        "eventTitle": "",
        "eventDivCd": "CED03",
        "eventTyCd": "",
        "orderReqCd": "ONGlist"
    }
    ua = UserAgent()
    headers = {
        'User-Agent': ua.random
    }
    response = requests.post(url, json=body, headers=headers)
    text = response.text
    soup = BeautifulSoup(text)

    eventBtn_list = soup.find_all("a", class_="eventBtn")
    event_list = [
        { 
            "event_no": x["data-no"],
            "event_name": x["title"],
            "period": x.find("p", class_="date").get_text().replace("\t", "").replace("\n", "")
        }
        for x in eventBtn_list
    ]
    event_df = pd.DataFrame(event_list)
    goods_info = [get_MB_GoodsInfo(x) for x in event_df['event_no']]
    event_df = pd.concat([event_df, pd.json_normalize(goods_info)], axis=1)
    return event_df


def get_MB_GoodsInfo(event_idx):
    url = f"https://www.megabox.co.kr/event/detail?eventNo={event_idx}"
    ua = UserAgent()
    headers = {
        'User-Agent': ua.random
    }
    response = requests.post(url, headers=headers)
    text = response.text
    soup = BeautifulSoup(text)
    goods_tag = soup.find("button", attrs={"id":"btnSelectGoodsStock"})
    if goods_tag is None:
        goods_info = None
    else:
        goods_info = {
            "data-nm": goods_tag["data-nm"],
            "data-pn": goods_tag["data-pn"]
        }
    return goods_info

In [81]:
event_df = get_MB_EventList()
event_df[~event_df['data-pn'].isna()].sort_values('data-pn')

,event_no,event_name,period,data-nm,data-pn
14,15264,메가박스 오리지널 티켓 No.107 <쿵푸팬더4> 상세보기,2024.04.10 ~ 2024.05.08,쿵푸팬더4 오리지널 티켓,FG000128
11,15275,메가박스 오리지널 티켓 No.108 <골드핑거> 상세보기,2024.04.13 ~ 2024.05.12,골드핑거 오리지널 티켓,FG000132
9,15325,메가박스 오리지널 티켓 No.109 <범죄도시4> 상세보기,2024.04.24 ~ 2024.05.24,범죄도시4 오리지널 티켓,FG000137


In [94]:
def get_MB_Theaters(goodsNo):
    url = "https://www.megabox.co.kr/on/oh/ohe/Event/selectGoodsStockPrco.do"
    body = {
        "goodsNo": goodsNo
    }
    ua = UserAgent()
    headers = {
        'User-Agent': ua.random
    }
    response = requests.post(url, data=body, headers=headers)
    text = response.text
    soup = BeautifulSoup(text)
    brch_list = soup.find_all("li", class_="brch")
    theater_list = [ 
        { 
            "theater": x.find("a").get_text(),
            "status": x.find("span").get_text()        
        }
        for x in brch_list
    ]
    Theaters = pd.DataFrame(theater_list)
    return Theaters

In [98]:
def Get_MB_GoodsStatus(goods_filter=""):
    event_df = get_MB_EventList()
    event_df = event_df[~event_df['data-pn'].isna()].sort_values('data-pn')
    event_df = event_df[
        (event_df["data-nm"].str.contains(goods_filter))
    ]
    GoodsStatus = pd.DataFrame()
    for idx in event_df.index:
        goodsNo = event_df.loc[idx]["data-pn"]
        Theaters = get_MB_Theaters(goodsNo)
        Theaters["event_name"] =  event_df.loc[idx]["event_name"]
        Theaters["data-nm"] =  event_df.loc[idx]["data-nm"]
        Theaters["period"] =  event_df.loc[idx]["period"]
        GoodsStatus = pd.concat([GoodsStatus, Theaters], ignore_index=True)
    return GoodsStatus

In [99]:
Get_MB_GoodsStatus()

,theater,status,event_name,data-nm,period
0,강남,소진,메가박스 오리지널 티켓 No.107 <쿵푸팬더4> 상세보기,쿵푸팬더4 오리지널 티켓,2024.04.10 ~ 2024.05.08
1,강남대로(씨티)_영업종료,소진,메가박스 오리지널 티켓 No.107 <쿵푸팬더4> 상세보기,쿵푸팬더4 오리지널 티켓,2024.04.10 ~ 2024.05.08
2,강동,소진,메가박스 오리지널 티켓 No.107 <쿵푸팬더4> 상세보기,쿵푸팬더4 오리지널 티켓,2024.04.10 ~ 2024.05.08
3,군자,소진,메가박스 오리지널 티켓 No.107 <쿵푸팬더4> 상세보기,쿵푸팬더4 오리지널 티켓,2024.04.10 ~ 2024.05.08
4,더 부티크 목동현대백화점,소진,메가박스 오리지널 티켓 No.107 <쿵푸팬더4> 상세보기,쿵푸팬더4 오리지널 티켓,2024.04.10 ~ 2024.05.08
...,...,...,...,...,...
308,원주혁신,소진,메가박스 오리지널 티켓 No.109 <범죄도시4> 상세보기,범죄도시4 오리지널 티켓,2024.04.24 ~ 2024.05.24
309,춘천석사,소진,메가박스 오리지널 티켓 No.109 <범죄도시4> 상세보기,범죄도시4 오리지널 티켓,2024.04.24 ~ 2024.05.24
310,제주삼화,소량보유,메가박스 오리지널 티켓 No.109 <범죄도시4> 상세보기,범죄도시4 오리지널 티켓,2024.04.24 ~ 2024.05.24
311,제주서귀포,보유,메가박스 오리지널 티켓 No.109 <범죄도시4> 상세보기,범죄도시4 오리지널 티켓,2024.04.24 ~ 2024.05.24


In [84]:
# 이벤트 목록에 없는 굿즈 정보 확인

for i in range(120, 145):
    
    goodsNo = "FG000"+str(i)
a
    url = "https://www.megabox.co.kr/on/oh/ohe/Event/selectGoodsStockPrco.do"
    body = {
        "goodsNo": goodsNo
    }
    ua = UserAgent()
    headers = {
        'User-Agent': ua.random
    }
    response = requests.post(url, data=body, headers=headers)
    text = response.text
    soup = BeautifulSoup(text)
    # brch_list = soup.find_all("li", class_="brch")
    # theater_list = [ 
    #     { 
    #         "theater": x.find("a").get_text(),
    #         "status": x.find("span").get_text()        
    #     }
    #     for x in brch_list
    # ]

    print(goodsNo, soup.find("div", class_="tit").get_text())
    # theater_list

FG000120 비키퍼 Dolby 포스터
FG000121 영웅본색 포스터
FG000122 기동전사 건담 SEED FREEDOM Dolby 포스터
FG000123 기동전사 건담 SEED FREEDOM MX4D 포스터
FG000124 남은 인생 10년 드로잉 카드
FG000125 오멘: 저주의 시작 A3 포스터
FG000126 쿵푸팬더4 Dolby 포스터
FG000127 로봇 드림 드로잉 카드
FG000128 쿵푸팬더4 오리지널 티켓
FG000129 8월의 크리스마스 인화사진
FG000130 
FG000131 
FG000132 골드핑거 오리지널 티켓
FG000133 듄 Dolby 포스터
FG000134 듄 MX4D 포스터
FG000135 고스트버스터즈 드로잉카드
FG000136 레옹 드로잉카드
FG000137 범죄도시4 오리지널 티켓
FG000138 혹성탈출 새로운 시대 Dolby 포스터
FG000139 쇼생크 탈출 오리지널티켓
FG000140 
FG000141 
FG000142 
FG000143 
FG000144 
